## Running OpenMM simulations

In [1]:
import os
from sys import stdout

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [2]:
root_dir_pdbs = '../data/pdbs_mutated_cleaned/'
root_dir_openmm = '../data/openmm_output/'
pdb_list = os.listdir(root_dir_pdbs)

In [ ]:
unsimulatables = []

for ele in pdb_list:    
    try:
        pdb = PDBFile(root_dir_pdbs + f'{ele}')
        forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
        # changed nonbondedMethod to NoCutoff from PME
        system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff,
                nonbondedCutoff=1*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
        simulation = Simulation(pdb.topology, system, integrator)
        simulation.context.setPositions(pdb.positions)
        simulation.minimizeEnergy()
    #     simulation.reporters.append(PDBReporter(root_dir_openmm + f'{ele[:-4]}', 1000))
        simulation.reporters.append(StateDataReporter(root_dir_openmm + f'{ele[:-4]}', 1000, step=True, time=True, 
                                                      potentialEnergy=True, kineticEnergy=True, totalEnergy=True, 
                                                      temperature=True, volume=True, density=True)) 
        simulation.step(20000)
        
    except:
        print(f"could not simulate {ele}")
        unsimulatables.append(ele)

could not simulate 3QHY_A_B_WB13A.pdb
